In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from collections import Counter
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns


import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['subbmission.csv', 'submission.csv', 'test.csv', 'train.csv']


In [6]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
submission = pd.read_csv('../input/submission.csv')


In [7]:
test['交易年月日']

0       0106-08-12 00:00:00
1       0104-12-10 00:00:00
2       0105-01-19 00:00:00
3       0105-05-24 00:00:00
4       0105-04-03 00:00:00
               ...         
1495    0104-06-12 00:00:00
1496    0105-12-27 00:00:00
1497    0104-06-21 00:00:00
1498    0103-11-20 00:00:00
1499    0106-09-10 00:00:00
Name: 交易年月日, Length: 1500, dtype: object

In [8]:
train['price_per_ping'] = train['單價(元/平方公尺)']
train['price_per_ping']

0         60870.0
1        117550.0
2        231650.0
3        127419.0
4        272743.0
           ...   
69165    140892.0
69166     15810.0
69167    238095.0
69168    189006.0
69169    355957.0
Name: price_per_ping, Length: 69170, dtype: float64

In [9]:
showing_cols = [
    '主要建材',
    '主要用途', '交易年月日', '交易標的', '交易筆棟數', '備註', '土地區段位置/建物區段門牌',
    '土地移轉總面積(平方公尺)', '建物型態', '建物現況格局-廳', '建物現況格局-房', '建物現況格局-衛',
    '建物現況格局-隔間', '建物移轉總面積(平方公尺)', '建築完成年月', '有無管理組織', '移轉層次',
    '總樓層數', '車位移轉總面積(平方公尺)', '車位類別', '都市土地使用分區', '鄉鎮市區',
    '非都市土地使用分區', 
    'num_of_bus_stations_in_100m', 'income_avg', 'income_var', 
    'location_type', 'low_use_electricity',
    'nearest_tarin_station', 'nearest_tarin_station_distance',
    'lat', 'lng', 'price_per_ping'
]
train = train[showing_cols]
train.info()
# test = test[showing_cols]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69170 entries, 0 to 69169
Data columns (total 33 columns):
主要建材                              62111 non-null object
主要用途                              60013 non-null object
交易年月日                             69170 non-null object
交易標的                              69170 non-null object
交易筆棟數                             69170 non-null object
備註                                31058 non-null object
土地區段位置/建物區段門牌                     69170 non-null object
土地移轉總面積(平方公尺)                     69170 non-null float64
建物型態                              69170 non-null object
建物現況格局-廳                          69170 non-null int64
建物現況格局-房                          69170 non-null int64
建物現況格局-衛                          69170 non-null int64
建物現況格局-隔間                         69170 non-null object
建物移轉總面積(平方公尺)                     69170 non-null float64
建築完成年月                            54798 non-null float64
有無管理組織                            69170 non-null ob

In [10]:
train.describe()

,土地移轉總面積(平方公尺),建物現況格局-廳,建物現況格局-房,建物現況格局-衛,建物移轉總面積(平方公尺),建築完成年月,車位移轉總面積(平方公尺),num_of_bus_stations_in_100m,income_avg,income_var,nearest_tarin_station_distance,lat,lng,price_per_ping
count,69170.000000,69170.000000,69170.000000,69170.000000,69170.000000,5.479800e+04,69170.000000,69166.000000,68808.000000,68808.000000,69166.000000,69166.000000,69166.000000,6.917000e+04
mean,57.365397,1.324967,2.099104,1.331907,134.382342,8.511429e+05,12.091376,2.237487,1298.164661,230.149715,623.246573,25.055852,121.543798,1.887246e+05
std,567.006678,0.937209,1.844211,1.279941,472.970544,1.557311e+05,71.567118,1.576150,498.857620,250.098417,511.895384,0.036838,0.031904,2.774005e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000e+00,0.000000,0.000000,682.000000,81.630000,0.000000,24.965286,121.464657,0.000000e+00
25%,11.520000,1.000000,1.000000,1.000000,52.960000,7.111020e+05,0.000000,1.000000,1005.000000,114.610000,297.997594,25.032969,121.518965,1.248660e+05
50%,23.850000,2.000000,2.000000,1.000000,98.095000,8.508240e+05,0.000000,2.000000,1133.000000,150.760000,494.796560,25.051739,121.541236,1.697065e+05
75%,37.000000,2.000000,3.000000,2.000000,154.797500,1.010426e+06,0.000000,3.000000,1446.000000,245.290000,751.628239,25.077493,121.565418,2.244038e+05
max,102554.980000,80.000000,168.000000,174.000000,69125.530000,1.070130e+06,12188.160000,10.000000,6598.000000,2403.410000,6007.085560,25.172812,121.624223,6.268571e+07


In [11]:
train['price_per_ping'].describe()

count    6.917000e+04
mean     1.887246e+05
std      2.774005e+05
min      0.000000e+00
25%      1.248660e+05
50%      1.697065e+05
75%      2.244038e+05
max      6.268571e+07
Name: price_per_ping, dtype: float64

In [12]:
total= train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)

In [13]:
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_data.head(20)

,Total,Percent
非都市土地使用分區,69160,0.999855
車位類別,47720,0.689894
備註,38112,0.550990
建築完成年月,14372,0.207778
主要用途,9157,0.132384
移轉層次,7258,0.104930
總樓層數,7206,0.104178
主要建材,7059,0.102053
都市土地使用分區,619,0.008949
income_avg,362,0.005233


In [14]:
train['建築完成年月'].describe()

count    5.479800e+04
mean     8.511429e+05
std      1.557311e+05
min      8.000000e+00
25%      7.111020e+05
50%      8.508240e+05
75%      1.010426e+06
max      1.070130e+06
Name: 建築完成年月, dtype: float64

In [15]:
train['建築完成年月']

0              NaN
1              NaN
2        1031204.0
3         571130.0
4         940803.0
           ...    
69165    1060106.0
69166          NaN
69167     911029.0
69168     700731.0
69169     971127.0
Name: 建築完成年月, Length: 69170, dtype: float64

In [16]:
train['建築完成年月'] = train['建築完成年月'].fillna(train['建築完成年月'].mean())
test['建築完成年月'] = test['建築完成年月'].fillna(test['建築完成年月'].mean())

In [17]:
train['建築完成年月'].describe()

count    6.917000e+04
mean     8.511429e+05
std      1.386111e+05
min      8.000000e+00
25%      7.311060e+05
50%      8.511429e+05
75%      9.801060e+05
max      1.070130e+06
Name: 建築完成年月, dtype: float64

In [18]:
train['建築完成年月']

0        8.511429e+05
1        8.511429e+05
2        1.031204e+06
3        5.711300e+05
4        9.408030e+05
             ...     
69165    1.060106e+06
69166    8.511429e+05
69167    9.110290e+05
69168    7.007310e+05
69169    9.711270e+05
Name: 建築完成年月, Length: 69170, dtype: float64

In [19]:
train['交易年月日'].loc[5][:4]

'0104'

In [20]:
train['交易年月日'].astype(str).apply(lambda x:int(x[:4]))

0        105
1        104
2        104
3        104
4        106
        ... 
69165    106
69166    104
69167    106
69168    106
69169    104
Name: 交易年月日, Length: 69170, dtype: int64

In [21]:
train['deal_year'] = train['交易年月日'].astype(str).apply(lambda x:int(x[:4]))
test['deal_year'] = test['交易年月日'].astype(str).apply(lambda x:int(x[:4]))

In [22]:
total= train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)

In [23]:
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_data.head(20)

,Total,Percent
非都市土地使用分區,69160,0.999855
車位類別,47720,0.689894
備註,38112,0.550990
主要用途,9157,0.132384
移轉層次,7258,0.104930
總樓層數,7206,0.104178
主要建材,7059,0.102053
都市土地使用分區,619,0.008949
income_avg,362,0.005233
low_use_electricity,362,0.005233


In [24]:
train= train.drop((missing_data[missing_data['Total'] > 20]).index,1)
train.isnull().sum()

交易年月日                             0
交易標的                              0
交易筆棟數                             0
土地區段位置/建物區段門牌                     0
土地移轉總面積(平方公尺)                     0
建物型態                              0
建物現況格局-廳                          0
建物現況格局-房                          0
建物現況格局-衛                          0
建物現況格局-隔間                         0
建物移轉總面積(平方公尺)                     0
建築完成年月                            0
有無管理組織                            0
車位移轉總面積(平方公尺)                     0
鄉鎮市區                              0
num_of_bus_stations_in_100m       4
location_type                     4
nearest_tarin_station             4
nearest_tarin_station_distance    4
lat                               4
lng                               4
price_per_ping                    0
deal_year                         0
dtype: int64

In [25]:
train[train.isnull().any(axis=1)]

,交易年月日,交易標的,交易筆棟數,土地區段位置/建物區段門牌,土地移轉總面積(平方公尺),建物型態,建物現況格局-廳,建物現況格局-房,建物現況格局-衛,建物現況格局-隔間,...,車位移轉總面積(平方公尺),鄉鎮市區,num_of_bus_stations_in_100m,location_type,nearest_tarin_station,nearest_tarin_station_distance,lat,lng,price_per_ping,deal_year
10958,0104-09-25 00:00:00,土地,土地2建物0車位0,雙連段一小段481~510地號,7.75,其他,0,0,0,有,...,0.0,大同區,NaN,NaN,NaN,NaN,NaN,NaN,568149.0,104
22713,0105-12-23 00:00:00,土地,土地4建物0車位0,雙連段一小段571~600地號,114.80,其他,0,0,0,有,...,0.0,大同區,NaN,NaN,NaN,NaN,NaN,NaN,635244.0,105
58874,0103-05-02 00:00:00,土地,土地3建物0車位0,雙連段一小段571~600地號,311.00,其他,0,0,0,有,...,0.0,大同區,NaN,NaN,NaN,NaN,NaN,NaN,637814.0,103
61577,0106-12-07 00:00:00,土地,土地4建物0車位0,雙連段一小段571~600地號,328.00,其他,0,0,0,有,...,0.0,大同區,NaN,NaN,NaN,NaN,NaN,NaN,609756.0,106


In [26]:
train[train.duplicated(subset=['交易年月日','土地區段位置/建物區段門牌'])]

,交易年月日,交易標的,交易筆棟數,土地區段位置/建物區段門牌,土地移轉總面積(平方公尺),建物型態,建物現況格局-廳,建物現況格局-房,建物現況格局-衛,建物現況格局-隔間,...,車位移轉總面積(平方公尺),鄉鎮市區,num_of_bus_stations_in_100m,location_type,nearest_tarin_station,nearest_tarin_station_distance,lat,lng,price_per_ping,deal_year
232,0104-04-29 00:00:00,房地(土地+建物)+車位,土地1建物1車位2,臺北市北投區磺港路121~150號,32.23,住宅大樓(11層含以上有電梯),2,4,2,有,...,62.82,北投區,0.0,RANGE_INTERPOLATED,奇岩站,445.169279,25.129573,121.503228,143170.0,104
327,0104-05-23 00:00:00,房地(土地+建物)+車位,土地1建物1車位1,臺北市北投區三合街一段82巷61~90號,21.26,華廈(10層含以下有電梯),2,2,1,有,...,43.59,北投區,2.0,RANGE_INTERPOLATED,奇岩站,471.169062,25.129118,121.504303,206092.0,104
543,0105-05-16 00:00:00,房地(土地+建物),土地1建物1車位0,臺北市萬華區康定路91~120號,3.53,套房(1房1廳1衛),1,1,1,有,...,0.00,萬華區,4.0,ROOFTOP,龍山寺站,550.927268,25.039796,121.502070,195548.0,105
627,0106-11-28 00:00:00,房地(土地+建物)+車位,土地1建物1車位1,臺北市北投區文林北路75巷61~90號,47.73,住宅大樓(11層含以上有電梯),2,3,2,有,...,43.74,北投區,0.0,RANGE_INTERPOLATED,明德站,588.652625,25.105111,121.515976,64146.0,106
630,0105-05-16 00:00:00,房地(土地+建物)+車位,土地1建物1車位1,臺北市萬華區康定路91~120號,5.72,住宅大樓(11層含以上有電梯),2,2,1,有,...,33.70,萬華區,4.0,ROOFTOP,龍山寺站,550.927268,25.039796,121.502070,211677.0,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69147,0104-12-03 00:00:00,土地,土地3建物0車位0,新洲美段1~30地號,77.41,其他,0,0,0,有,...,0.00,北投區,2.0,APPROXIMATE,明德站,1958.214739,25.101275,121.501789,158403.0,104
69157,0103-12-31 00:00:00,房地(土地+建物)+車位,土地1建物1車位2,臺北市南港區經貿二路235巷1~30號,57.08,住宅大樓(11層含以上有電梯),2,3,2,有,...,55.14,南港區,4.0,RANGE_INTERPOLATED,南港軟體園區站,104.595160,25.060303,121.614971,264610.0,103
69161,0101-04-27 00:00:00,房地(土地+建物)+車位,土地1建物1車位2,臺北市北投區裕民六路121~150號,18.87,住宅大樓(11層含以上有電梯),2,3,2,有,...,55.22,北投區,0.0,RANGE_INTERPOLATED,石牌站,186.802582,25.113792,121.517229,227775.0,101
69165,0106-09-30 00:00:00,房地(土地+建物)+車位,土地1建物1車位1,臺北市文山區下崙路11巷1~30號,33.01,華廈(10層含以下有電梯),2,3,2,有,...,0.00,文山區,2.0,ROOFTOP,萬芳醫院站,1463.742646,24.986362,121.560260,140892.0,106


In [27]:
train = train.drop_duplicates(subset=['交易年月日','土地區段位置/建物區段門牌'])

In [28]:
train = train.dropna()
train.isnull().any()

交易年月日                             False
交易標的                              False
交易筆棟數                             False
土地區段位置/建物區段門牌                     False
土地移轉總面積(平方公尺)                     False
建物型態                              False
建物現況格局-廳                          False
建物現況格局-房                          False
建物現況格局-衛                          False
建物現況格局-隔間                         False
建物移轉總面積(平方公尺)                     False
建築完成年月                            False
有無管理組織                            False
車位移轉總面積(平方公尺)                     False
鄉鎮市區                              False
num_of_bus_stations_in_100m       False
location_type                     False
nearest_tarin_station             False
nearest_tarin_station_distance    False
lat                               False
lng                               False
price_per_ping                    False
deal_year                         False
dtype: bool

In [29]:
# train.describe()

In [30]:
train['土地移轉總面積(平方公尺)'].sort_values(ascending=False)

13178    102554.98
32874     37828.00
62835     26177.76
7853      21765.27
62309     21660.29
           ...    
6242          0.00
43055         0.00
41614         0.00
13235         0.00
53618         0.00
Name: 土地移轉總面積(平方公尺), Length: 61678, dtype: float64

In [31]:
col = ['price_per_ping','土地移轉總面積(平方公尺)','建物移轉總面積(平方公尺)','建物現況格局-廳','建物現況格局-房','建物現況格局-衛','建築完成年月','deal_year']

In [32]:
# train[col].sort_values(by=['price_per_ping'],ascending=False).tail(20)


In [33]:
train[col][train['price_per_ping']<10000].count()
train[col][train['price_per_ping']>10000000].count()


price_per_ping    1
土地移轉總面積(平方公尺)     1
建物移轉總面積(平方公尺)     1
建物現況格局-廳          1
建物現況格局-房          1
建物現況格局-衛          1
建築完成年月            1
deal_year         1
dtype: int64

In [34]:
train = train[(train['price_per_ping'] <3000000) & (train['price_per_ping'] >10000)]
# sns.distplot(train['price_per_ping']);

In [35]:
train = train[(train['土地移轉總面積(平方公尺)'] > 0) &(train['土地移轉總面積(平方公尺)']< 500)]
# sns.distplot(train['土地移轉總面積(平方公尺)']);

In [36]:
train = train[(train['建物移轉總面積(平方公尺)'] > 0) &(train['建物移轉總面積(平方公尺)']< 500)]
#sns.distplot(train['建物移轉總面積(平方公尺)']);                   

In [37]:
train[train['建物現況格局-廳']> 10].count() / train['建物現況格局-廳'].count()
train = train[train['建物現況格局-廳']< 10]
# sns.distplot(train['建物現況格局-廳']);

In [38]:
train[train['建物現況格局-房']> 20].count() / train['建物現況格局-房'].count()
train = train[train['建物現況格局-房']< 20]

In [39]:
train = train[train['建物現況格局-衛']< 20]

In [40]:
train= train[train['建物現況格局-衛']< 20]

In [41]:
train.reset_index(drop=True, inplace=True)
X_columns = [col for col in train[col]]
X_columns.remove('price_per_ping')
X_columns

['土地移轉總面積(平方公尺)',
 '建物移轉總面積(平方公尺)',
 '建物現況格局-廳',
 '建物現況格局-房',
 '建物現況格局-衛',
 '建築完成年月',
 'deal_year']

* # OneHotEnconder 

In [42]:
from sklearn.preprocessing import OneHotEncoder
train['鄉鎮市區']
a = train['鄉鎮市區'].astype(str).values
b = test['鄉鎮市區'].astype(str).values
categories = sorted(list(set(a)))
enc = OneHotEncoder(categories=[categories], handle_unknown='ignore')
train_鄉鎮市區onehot = enc.fit_transform(a.reshape(-1,1)).toarray()
test_鄉鎮市區onehot = enc.fit_transform(b.reshape(-1,1)).toarray()

# Prediction

In [43]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier  ## KNN
from sklearn.linear_model import LogisticRegressionCV  ## logistic regression
from sklearn.tree import DecisionTreeClassifier  ## decision tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import xgboost
from xgboost import XGBClassifier

In [44]:
from sklearn.model_selection import train_test_split

X = train[X_columns].values
X = np.concatenate([X, train_鄉鎮市區onehot], axis=1)
Y = train[['price_per_ping']].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1212)

In [ ]:
def get_accuracy(clf):
    #=============your works starts===============#
    clf = clf()
    clf =  clf.fit(X_train, Y_train.ravel())
    y_pred = clf.predict(X_test)
    accuracy = (str(sum(Y_test == y_pred)/Y_test.shape[0]))
    #==============your works ends================#
    return accuracy

#print('SVM: ', get_accuracy(SVC))
#print('DecisionTree: ', get_accuracy(DecisionTreeClassifier))
#print('RandomForest: ', get_accuracy(RandomForestClassifier))
#print('AdaBoost: ', get_accuracy(AdaBoostClassifier))  ## Boosting的演算法
print('XGB: ', get_accuracy(XGBClassifier))


In [ ]:
# Set our parameters for xgboost
params = {}
# 請填入以下參數: 
# 目標函數: 二元分類
# 評價函數: logloss
# 學習速度: 0.04
# 最大深度: 5
#=============your works starts===============#
params['objective'] = 'reg:squarederror'
params['eval_metric'] = ['logloss', 'error', 'error@0.3']    
params['eta'] = 0.04       #learning rate: 學習時所增加值的幅度, 太大會有過度修正, 來回跳動的問題, 太小學習慢
params['max_depth'] = 5    
#==============your works ends================#

d_train = xgboost.DMatrix(X_train, label=Y_train)
d_test = xgboost.DMatrix(X_test, label=Y_test)

watchlist = [(d_train, 'train'), (d_test, 'test')]

bst = xgboost.train(params, d_train, 1000, watchlist, early_stopping_rounds=100, verbose_eval=0)  
# verbose_eval 看跑的過程
y_pred = bst.predict(xgboost.DMatrix(X_test))

print("Accuracy: ", str(sum(Y_test == (y_pred > 0.5))/Y_test.shape[0]))



In [ ]:
my_submission = pd.DataFrame({'index':submission.index,'price_per_ping': y_pred})
my_submission.to_csv('submission.csv', index=False)
print(predictions)

In [ ]:
#X = test[X_columns].values

#predictions = np.expm1(xgb.predict(X))
#my_submission = pd.DataFrame({'index':submission.index,'price_per_ping': predictions})
#my_submission.to_csv('submission.csv', index=False)
#print(predictions)

In [ ]:
# import xgboost
# from sklearn.metrics import r2_score

##gpuconf = { 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor' }

# xgb = xgboost.XGBRegressor(
#    max_depth=10,
#    n_estimators=2000,
#    learning_rate=0.01,
#     n_estimators=1000,
##     learning_rate=0.08,
#     gamma=0,
#    subsample=0.5,
#    n_jobs=4,
#    objective='reg:squarederror',
#    alpha=0.0001,
#    verbosity=3,
#    #**gpuconf
#)

# xgb.fit(X_train, Y_train)
# predictions = xgb.predict(X_test)
# print(f'R2 Score: {r2_score(Y_test, predictions)}')

import xgboost
from sklearn.metrics import r2_score

#gpuconf = { 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor' }

xgb = xgboost.XGBRegressor(
    max_depth=10,
    n_estimators=2000,
    learning_rate=0.01,
#     n_estimators=1000,
#     learning_rate=0.08,
    gamma=0,
    subsample=0.5,
    n_jobs=4,
    objective='reg:squarederror',
    alpha=0.0001,
    verbosity=3,
    #**gpuconf
)

xgb.fit(X_train, Y_train)
predictions = xgb.predict(X_test)
print(f'R2 Score: {r2_score(Y_test, predictions)}')